From the root directory of the repo:

`pip install libs/z534`

`pip install libs/SemDiff`

In [18]:
import pymongo
import pandas as pd
import z534
import SemDiff as sd

Requires having Mongodb running with a database called 'yelp' and a collection called 'businesses' containing all of the businesses from the yelp dataset. 

In [19]:
db = z534.get_db()

In [20]:
db.businesses.find_one({'list_of_categories': 'Restaurants'})

{'_id': ObjectId('5c9f917de7e30ee4bd23d8b1'),
 'business_id': 'QXAEGFB4oINsVuTFxEYKFQ',
 'name': 'Emerald Chinese Restaurant',
 'address': '30 Eglinton Avenue W',
 'city': 'Mississauga',
 'state': 'ON',
 'postal_code': 'L5R 3E7',
 'latitude': 43.6054989743,
 'longitude': -79.652288909,
 'stars': 2.5,
 'review_count': 128,
 'is_open': 1,
 'attributes': {'RestaurantsReservations': 'True',
  'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  'Caters': 'True',
  'NoiseLevel': "u'loud'",
  'RestaurantsTableService': 'True',
  'RestaurantsTakeOut': 'True',
  'RestaurantsPriceRange2': '2',
  'OutdoorSeating': 'False',
  'BikeParking': 'False',
  'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': Tr

In [21]:
db.businesses.find_one({'list_of_categories': 'Hotels'})

{'_id': ObjectId('5c9f917de7e30ee4bd23d8df'),
 'business_id': 'Jff5hTK1ZMLKS06sOSNWdA',
 'name': 'Radisson Hotel & Conference Centre Calgary Airport',
 'address': '6620 36 Street NE',
 'city': 'Calgary',
 'state': 'AB',
 'postal_code': 'T3J 4C8',
 'latitude': 51.1118915249,
 'longitude': -113.9804801532,
 'stars': 3.5,
 'review_count': 14,
 'is_open': 1,
 'attributes': {'RestaurantsPriceRange2': '2', 'WiFi': "'free'"},
 'categories': 'Hotels & Travel, Hotels, Event Planning & Services',
 'hours': None,
 'list_of_categories': ['Hotels & Travel',
  'Hotels',
  'Event Planning & Services']}

In [22]:
db.businesses.find_one({'list_of_categories': 'Shopping'})

{'_id': ObjectId('5c9f917de7e30ee4bd23d8af'),
 'business_id': 'HhyxOkGAM07SRYtlQ4wMFQ',
 'name': 'Queen City Plumbing',
 'address': '4209 Stuart Andrew Blvd, Ste F',
 'city': 'Charlotte',
 'state': 'NC',
 'postal_code': '28217',
 'latitude': 35.1900119,
 'longitude': -80.8872232,
 'stars': 4.0,
 'review_count': 4,
 'is_open': 1,
 'attributes': {'BusinessAcceptsBitcoin': 'False',
  'ByAppointmentOnly': 'True',
  'BusinessAcceptsCreditCards': 'True'},
 'categories': 'Plumbing, Shopping, Local Services, Home Services, Kitchen & Bath, Home & Garden, Water Heater Installation/Repair',
 'hours': {'Monday': '7:0-23:0',
  'Tuesday': '7:0-23:0',
  'Wednesday': '7:0-23:0',
  'Thursday': '7:0-23:0',
  'Friday': '7:0-23:0',
  'Saturday': '7:0-23:0',
  'Sunday': '7:0-23:0'},
 'list_of_categories': ['Plumbing',
  'Shopping',
  'Local Services',
  'Home Services',
  'Kitchen & Bath',
  'Home & Garden',
  'Water Heater Installation/Repair']}

In [23]:
restaurants_cursor = db.businesses.find({'list_of_categories': 'Restaurants'})
hotels_cursor = db.businesses.find({'list_of_categories': 'Hotels'})
store_cursor = db.businesses.find({'list_of_categories': 'Shopping'})

restaurants = []
hotels = []
stores = []

for item in restaurants_cursor:
    restaurants.append(item)
    
for item in hotels_cursor:
    hotels.append(item)
    
for item in store_cursor:
    stores.append(item)
    
print("Number of hotels: {}\nNumber of restaurants: {}\n Number of stores: {}" \
      .format(len(hotels), len(restaurants), len(stores)))

Number of hotels: 2625
Number of restaurants: 59371
 Number of stores: 31878


In [25]:
print('querying hotel reviews')
hotel_reviews = z534.mongo_utils.aggregate_reviews(db, business_category = 'Hotels')
print('querying restaurant reviews')
restaurant_reviews = z534.mongo_utils.aggregate_reviews(db, business_category = 'Restaurants')
print('querying shopping reviews')
store_reviews = z534.mongo_utils.aggregate_reviews(db, business_category = 'Shopping')

print("\nNumber of hotel reviews: {}\nNumber of restaurant reviews: {}\nNumber of store reviews: {}" \
      .format(len(hotel_reviews), len(restaurant_reviews), len(store_reviews)))

querying hotel reviews
querying restaurant reviews
querying shopping reviews

Number of hotel reviews: 231380
Number of restaurant reviews: 4159944
Number of store reviews: 468741


In [26]:
hotel_reviews_filt = hotel_reviews[:230000]
restaurant_reviews_filt = restaurant_reviews[:230000]
store_reviews_filt = store_reviews[:230000]

In [27]:
hotel_df = pd.DataFrame(hotel_reviews_filt)
restaurant_df = pd.DataFrame(restaurant_reviews_filt)
store_df = pd.DataFrame(store_reviews_filt)

In [28]:
import os

In [29]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

This needs to be the same directory as is specified in `generate_gensim_models_yelp.py`

In [32]:
DATA_DIR = '/Users/bsobolik/Documents/semantic_difference/data/'
ensure_dir(DATA_DIR)

In [33]:
hotel_df.to_csv(DATA_DIR + 'hotel_text.csv')
restaurant_df.to_csv(DATA_DIR + 'restaurant_text.csv')
store_df.to_csv(DATA_DIR + 'store_text.csv')

At this point, `generate_gensim_models_yelp.py` needs to be run to create the word vectors. 

In [34]:
# Now load the models that have already been trained
from gensim.models import Word2Vec
hotel_model = Word2Vec.load(DATA_DIR + "hotel_model.wv")
restaurant_model = Word2Vec.load(DATA_DIR + "restaurants_model.wv")
store_model = Word2Vec.load(DATA_DIR + "store_model.wv")

/Users/bsobolik/.pyenv/versions/3.6.4/envs/tensorflow/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [35]:
# repeat for a set of antonyms
antonyms = [
    ["good", "bad"],
    ['always', 'never'],
    ['higher', 'lower'],
    ['certain', 'impossible'],
    ['strong', 'weak'],
    ['very', 'slightly'],
    ['similar', 'unlike'],
    ['unexpected', 'expected'],
    ['outside', 'inside'],
    ['start', 'stop'],
    ['worst', 'best'],
    ['happy', 'sad'],
    ['everywhere', 'nowhere'],
    ['common', 'exceptional'],
    ['fast', 'slow'],
    ['average', 'poor'],
    ['man', 'woman']
    
]

words = ['customer', 'host', 'food', 'bed', 'cashier', 'parking', 'room', 'table', 'drink']
models = [hotel_model, restaurant_model]
models_names = ["hotel", "restaurant"]
hotel_list = []
restaurant_list = []

ant_names = []
word_list = []
for word in words:
    for ant in antonyms:
        hotel_list.append(sd.project_word_on_axis(hotel_model, word, ant, k = 3))
        restaurant_list.append(sd.project_word_on_axis(restaurant_model, word, ant, k = 3))

        word_list.append(word)
    
        ant_names.append("{}-{}".format(ant[0], ant[1]))

In [36]:
df = pd.DataFrame({'antonym': ant_names, 
                   'hotel': hotel_list, 
                   'restaurant': restaurant_list, 
                   'word': word_list}, 
                  index = ant_names)
df.head()

,antonym,hotel,restaurant,word
good-bad,good-bad,0.335567,0.374934,customer
always-never,always-never,-0.069485,0.058174,customer
higher-lower,higher-lower,-0.033332,-0.023100,customer
certain-impossible,certain-impossible,0.019778,-0.072814,customer
strong-weak,strong-weak,0.106488,-0.026580,customer


In [37]:
df.to_csv(DATA_DIR + 'yelp_semantic_word_axes.csv')

In [38]:
shared = sd.get_shared_words(hotel_model, restaurant_model, topn_words = 10000)
frames = []
for word in words:
    frames.append(sd.model_term_similarity(hotel_model, restaurant_model, word, shared))
    
hotel_restaurant_model = pd.concat(frames)

hotel_restaurant_model.to_csv(DATA_DIR + 'hotel_restaurant_interword_similarity.csv')

In [39]:
hotel_restaurant_model.head()

,word,x,y
0,customer,-0.258649,-0.166106
1,customer,-0.121688,-0.166807
2,customer,0.123599,0.215254
3,customer,-0.174325,-0.130121
4,customer,0.246405,0.253227
